In [5]:
import pandas as pd
from torch import nn
from torch.optim import SGD
from pandas_ml_common.utils.column_lagging_utils import lag_columns

from pandas_ml_utils import FeaturesAndLabels, RegressionSummary
from pandas_ml_utils_torch import PytorchNN, PytorchModel
from pandas_ml_utils_torch.merging_cross_folds import take_the_best


,Date,Open,High,Low,Close,Adj Close,Volume
6758,2019-11-29,314.859985,315.130005,314.059998,314.309998,314.309998,36592700
6759,2019-12-02,314.589996,314.660004,311.170013,311.640015,311.640015,75767800
6760,2019-12-03,308.649994,309.640015,307.130005,309.549988,309.549988,73941700
6761,2019-12-04,310.700012,312.119995,310.320007,311.459991,311.459991,49080000
6762,2019-12-05,312.230011,312.250000,310.580109,312.089996,312.089996,40781669


## Fist lets load some data

In [ ]:
df = pd.read_csv('SPY.csv')
df.tail()


## Fit Regularized Model
Now we just build a very naive model where we add an L1 and L2 penalization term. Don't expect the model to predict
anything useful

In [11]:
with df.model() as m:

    class Net(PytorchNN):

        def __init__(self):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(10, 4),
                nn.Tanh(),
                nn.Linear(4, 4),
                nn.Tanh(),
                nn.Linear(4, 1),
                nn.Tanh(),
            )

        def L1(self):
            # path to the parameters which should be regularized
            # the path is constructed from self.named_parameters() and allows the use of wildcards
            return {'net/0/**/weight': 0.02}

        def L2(self):
            return {
                'net/0/**/weight': 0.02,
                'net/2/**/weight': 0.05
            }

        def forward_training(self, x):
            return self.net(x)

    fit = m.fit(
        PytorchModel(
            FeaturesAndLabels(
                [lambda df: lag_columns(df["Close"].pct_change(), range(10))],
                [lambda df: df["Close"].pct_change().shift(-1)]),
            Net,
            nn.MSELoss,
            lambda params: SGD(params, lr=0.01, momentum=0.0),
            merge_cross_folds=take_the_best,
            summary_provider=RegressionSummary
        ),
        fold_epochs=100,
    )

fit




mean_gamma_deviance failed: (ValueError('Mean Tweedie deviance error with power=2 can only be used on strictly positive y and y_pred.'), 'error while calling <function mean_gamma_deviance at 0x7f82de50ef80>(FullArgSpec(args=[], varargs=\'args\', varkw=\'kwargs\', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={}))\n@_deprecate_positional_args\ndef mean_gamma_deviance(y_true, y_pred, *, sample_weight=None):\n    """Mean Gamma deviance regression loss.\n\n    Gamma deviance is equivalent to the Tweedie deviance with\n    the power parameter `power=2`. It is invariant to scaling of\n    the target variable, and measures relative errors.\n\n    Read more in the :ref:`User Guide <mean_tweedie_deviance>`.\n\n    Parameters\n    ----------\n    y_true : array-like of shape (n_samples,)\n        Ground truth (correct) target values. Requires y_true > 0.\n\n    y_pred : array-like of shape (n_samples,)\n        Estimated target values. Requires y_pred > 0.\n\n    sample_weight :